# Green Purchasing Behavior Cube - NoSQL Project

## Project Overview
This project implements a custom JSON parser and NoSQL database operations to analyze the relationship between consumer spending on sustainable foods and economic factors like income and jobs.

## Team: Individual Project (Shamik Basu)


## Part 1: Extended JSON Parser

Extending the sample code to handle:
- Arrays
- Nested objects and arrays
- Boolean and null values
- Complex JSON structures


In [1]:
# Extended JSON Parser Implementation
# Based on sample code, extended to handle arrays, nested structures, booleans, and null

import re

def parse_string(str):
    """Parse a string value from JSON"""
    str = str.lstrip()
    assert(str[0] == '"'), f"Expected '\"' but found '{str[0]}'"
    str = str[1:]  # skip the start quote
    
    # Handle escaped characters (basic support)
    mystr = ""
    i = 0
    while i < len(str):
        if str[i] == '\\' and i + 1 < len(str):
            # Handle escape sequences
            if str[i+1] == 'n':
                mystr += '\n'
                i += 2
            elif str[i+1] == 't':
                mystr += '\t'
                i += 2
            elif str[i+1] == '\\':
                mystr += '\\'
                i += 2
            elif str[i+1] == '"':
                mystr += '"'
                i += 2
            else:
                mystr += str[i]
                i += 1
        elif str[i] == '"':
            # End of string
            rest = str[i + 1:]
            return mystr, rest
        else:
            mystr += str[i]
            i += 1
    
    raise ValueError('Unterminated string')

def parse_number(str):
    """Parse a number (int or float) from JSON"""
    str = str.lstrip()
    
    chs = ''
    is_float = False
    i = 0
    for ch in str:
        if (ch.isdigit() or ch == '.' or ch == '-' or ch == '+' or ch == 'e' or ch == 'E'):
            if ch == '.':
                is_float = True
            chs += ch
            i += 1
        else:
            break
    
    if len(chs) == 0:
        raise ValueError('Expected number but found nothing')
    
    str = str[i:]
    value = float(chs) if is_float else int(chs)
    return value, str

def parse_boolean(str):
    """Parse boolean values (true/false)"""
    str = str.lstrip()
    if str.startswith('true'):
        return True, str[4:]
    elif str.startswith('false'):
        return False, str[5:]
    else:
        raise ValueError('Expected boolean but found something else')

def parse_null(str):
    """Parse null value"""
    str = str.lstrip()
    if str.startswith('null'):
        return None, str[4:]
    else:
        raise ValueError('Expected null but found something else')

def parse_colon(str):
    """Consume a colon ':'"""
    str = str.lstrip()
    assert(str[0] == ':'), f"Expected ':' but found '{str[0]}'"
    return str[1:]

def parse_value(str):
    """Parse any JSON value (object, array, string, number, boolean, null)"""
    str = str.lstrip()
    
    if len(str) == 0:
        raise ValueError('Unexpected end of string')
    
    if str[0] == '{':
        return parse_object(str)
    elif str[0] == '[':
        return parse_array(str)
    elif str[0] == '"':
        return parse_string(str)
    elif str[0] == '-' or str[0].isdigit():
        return parse_number(str)
    elif str.startswith('true') or str.startswith('false'):
        return parse_boolean(str)
    elif str.startswith('null'):
        return parse_null(str)
    else:
        raise ValueError(f'Unexpected character: {str[0]}')

def parse_object(str):
    """Parse a JSON object (dictionary) - extended to handle nested structures"""
    str = str.lstrip()
    assert(str[0] == '{'), f"Expected '{{' but found '{str[0]}'"
    str = str[1:]  # skip {
    
    obj = {}
    
    while True:
        str = str.lstrip()
        
        if len(str) == 0:
            raise ValueError('Expecting "}" but reached the end of string!')
        elif str[0] == '}':  # end of json object
            str = str[1:]  # consume '}'
            return obj, str
        elif str[0] == ',':
            str = str[1:]  # skip ','
        else:  # ready for a new key-value pair
            key, str = parse_string(str)
            str = parse_colon(str)  # skip colon
            value, str = parse_value(str)  # parse any type of value
            obj[key] = value

def parse_array(str):
    """Parse a JSON array (list) - handles nested structures"""
    str = str.lstrip()
    assert(str[0] == '['), f"Expected '[' but found '{str[0]}'"
    str = str[1:]  # skip [
    
    arr = []
    
    while True:
        str = str.lstrip()
        
        if len(str) == 0:
            raise ValueError('Expecting "]" but reached the end of string!')
        elif str[0] == ']':  # end of array
            str = str[1:]  # consume ']'
            return arr, str
        elif str[0] == ',':
            str = str[1:]  # skip ','
        else:  # ready for a new value
            value, str = parse_value(str)  # parse any type of value
            arr.append(value)

def json_load(json_str):
    """Main function to load JSON string into Python object"""
    json_str = json_str.strip()
    value, rest = parse_value(json_str)
    rest = rest.strip()
    if len(rest) > 0:
        raise ValueError(f'Unexpected content after JSON: {rest[:20]}')
    return value

# Test the extended parser
print("Testing Extended JSON Parser:")
print("=" * 50)

# Test 1: Simple object
test1 = '{"name": "john", "age": 25.3, "gender": "male"}'
result1 = json_load(test1)
print("Test 1 - Simple object:", result1)

# Test 2: Nested object
test2 = '{"person": {"name": "john", "age": 25}, "city": "LA"}'
result2 = json_load(test2)
print("Test 2 - Nested object:", result2)

# Test 3: Array
test3 = '[1, 2, 3, "hello", true, null]'
result3 = json_load(test3)
print("Test 3 - Array:", result3)

# Test 4: Array of objects
test4 = '[{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}]'
result4 = json_load(test4)
print("Test 4 - Array of objects:", result4)

# Test 5: Complex nested structure
test5 = '{"data": [{"county": "LA", "spend": 1000}, {"county": "NY", "spend": 2000}], "year": 2023}'
result5 = json_load(test5)
print("Test 5 - Complex nested:", result5)


Testing Extended JSON Parser:
Test 1 - Simple object: {'name': 'john', 'age': 25.3, 'gender': 'male'}
Test 2 - Nested object: {'person': {'name': 'john', 'age': 25}, 'city': 'LA'}
Test 3 - Array: [1, 2, 3, 'hello', True, None]
Test 4 - Array of objects: [{'id': 1, 'name': 'Alice'}, {'id': 2, 'name': 'Bob'}]
Test 5 - Complex nested: {'data': [{'county': 'LA', 'spend': 1000}, {'county': 'NY', 'spend': 2000}], 'year': 2023}


## Part 2: Collection/DataFrame Structure

Implementing a collection structure to store JSON documents (similar to MongoDB collections)


In [2]:
# Collection class to store JSON documents (similar to MongoDB collections)

class Collection:
    """A collection class to store and manipulate JSON documents"""
    
    def __init__(self, name):
        self.name = name
        self.documents = []  # List of dictionaries (JSON objects)
    
    def insert(self, document):
        """Insert a document (dictionary) into the collection"""
        if isinstance(document, dict):
            self.documents.append(document)
        else:
            raise TypeError("Document must be a dictionary")
    
    def insert_many(self, documents):
        """Insert multiple documents into the collection"""
        for doc in documents:
            self.insert(doc)
    
    def __len__(self):
        return len(self.documents)
    
    def __getitem__(self, index):
        return self.documents[index]
    
    def __iter__(self):
        return iter(self.documents)
    
    def __repr__(self):
        return f"Collection(name='{self.name}', documents={len(self.documents)})"
    
    def to_list(self):
        """Return all documents as a list"""
        return self.documents.copy()

# Test Collection
print("Testing Collection Class:")
print("=" * 50)

collection = Collection("test_collection")
collection.insert({"name": "Alice", "age": 30})
collection.insert({"name": "Bob", "age": 25})
print(f"Collection: {collection}")
print(f"Documents: {collection.to_list()}")


Testing Collection Class:
Collection: Collection(name='test_collection', documents=2)
Documents: [{'name': 'Alice', 'age': 30}, {'name': 'Bob', 'age': 25}]


## Part 3: Core Operations

Implementing filtering, projection, group by, aggregation, and join operations


In [3]:
# Operation 1: Filtering
def filter_collection(collection, condition_func):
    """
    Filter documents in a collection based on a condition function
    
    Args:
        collection: Collection object
        condition_func: Function that takes a document and returns True/False
    
    Returns:
        New Collection with filtered documents
    """
    filtered = Collection(f"{collection.name}_filtered")
    for doc in collection:
        if condition_func(doc):
            filtered.insert(doc.copy())
    return filtered

# Operation 2: Projection
def project_collection(collection, fields):
    """
    Project (select) specific fields from documents
    
    Args:
        collection: Collection object
        fields: List of field names to select
    
    Returns:
        New Collection with projected documents
    """
    projected = Collection(f"{collection.name}_projected")
    for doc in collection:
        new_doc = {}
        for field in fields:
            if field in doc:
                new_doc[field] = doc[field]
        projected.insert(new_doc)
    return projected

# Operation 3: Group By
def group_by(collection, group_key):
    """
    Group documents by a key
    
    Args:
        collection: Collection object
        group_key: Field name to group by
    
    Returns:
        Dictionary where keys are group values and values are lists of documents
    """
    groups = {}
    for doc in collection:
        if group_key in doc:
            key_value = doc[group_key]
            if key_value not in groups:
                groups[key_value] = []
            groups[key_value].append(doc)
    return groups

# Operation 4: Aggregation
def aggregate(collection, group_key, agg_field, agg_func):
    """
    Group by a key and apply an aggregation function to a field
    
    Args:
        collection: Collection object
        group_key: Field name to group by
        agg_field: Field name to aggregate
        agg_func: Aggregation function (e.g., 'sum', 'avg', 'max', 'min', 'count')
    
    Returns:
        List of dictionaries with group_key and aggregated value
    """
    groups = group_by(collection, group_key)
    results = []
    
    for key_value, docs in groups.items():
        values = [doc[agg_field] for doc in docs if agg_field in doc]
        
        if len(values) == 0:
            continue
            
        if agg_func == 'sum':
            agg_value = sum(values)
        elif agg_func == 'avg':
            agg_value = sum(values) / len(values)
        elif agg_func == 'max':
            agg_value = max(values)
        elif agg_func == 'min':
            agg_value = min(values)
        elif agg_func == 'count':
            agg_value = len(values)
        else:
            raise ValueError(f"Unknown aggregation function: {agg_func}")
        
        results.append({group_key: key_value, f"{agg_func}({agg_field})": agg_value})
    
    return results

# Operation 5: Join
def join_collections(collection1, collection2, key1, key2):
    """
    Join two collections on specified keys
    
    Args:
        collection1: First Collection object
        collection2: Second Collection object
        key1: Key in collection1 to join on
        key2: Key in collection2 to join on
    
    Returns:
        New Collection with joined documents
    """
    joined = Collection(f"{collection1.name}_join_{collection2.name}")
    
    # Build index on collection2 for faster lookup
    index = {}
    for doc2 in collection2:
        if key2 in doc2:
            key_value = doc2[key2]
            if key_value not in index:
                index[key_value] = []
            index[key_value].append(doc2)
    
    # Perform join
    for doc1 in collection1:
        if key1 in doc1:
            key_value = doc1[key1]
            if key_value in index:
                for doc2 in index[key_value]:
                    # Merge documents
                    merged = doc1.copy()
                    # Add fields from doc2, avoiding conflicts by prefixing
                    for k, v in doc2.items():
                        if k != key2:  # Don't duplicate the join key
                            if k in merged:
                                merged[f"{collection2.name}_{k}"] = v
                            else:
                                merged[k] = v
                    joined.insert(merged)
    
    return joined

# Test operations
print("Testing Core Operations:")
print("=" * 50)

# Create test collection
test_coll = Collection("test")
test_coll.insert_many([
    {"county": "LA", "spend": 1000, "year": 2023},
    {"county": "NY", "spend": 2000, "year": 2023},
    {"county": "LA", "spend": 1500, "year": 2024},
    {"county": "NY", "spend": 2500, "year": 2024},
])

print("Original collection:")
for doc in test_coll:
    print(f"  {doc}")

# Test filtering
print("\n1. Filtering (spend > 1500):")
filtered = filter_collection(test_coll, lambda doc: doc.get("spend", 0) > 1500)
for doc in filtered:
    print(f"  {doc}")

# Test projection
print("\n2. Projection (county, spend):")
projected = project_collection(test_coll, ["county", "spend"])
for doc in projected:
    print(f"  {doc}")

# Test group by
print("\n3. Group by county:")
groups = group_by(test_coll, "county")
for key, docs in groups.items():
    print(f"  {key}: {len(docs)} documents")

# Test aggregation
print("\n4. Aggregation (sum of spend by county):")
agg_result = aggregate(test_coll, "county", "spend", "sum")
for result in agg_result:
    print(f"  {result}")

# Test join
print("\n5. Join:")
coll1 = Collection("coll1")
coll1.insert_many([
    {"county": "LA", "population": 10000000},
    {"county": "NY", "population": 8000000},
])

coll2 = Collection("coll2")
coll2.insert_many([
    {"county_code": "LA", "unemployment": 5.2},
    {"county_code": "NY", "unemployment": 4.8},
])

joined = join_collections(coll1, coll2, "county", "county_code")
for doc in joined:
    print(f"  {doc}")


Testing Core Operations:
Original collection:
  {'county': 'LA', 'spend': 1000, 'year': 2023}
  {'county': 'NY', 'spend': 2000, 'year': 2023}
  {'county': 'LA', 'spend': 1500, 'year': 2024}
  {'county': 'NY', 'spend': 2500, 'year': 2024}

1. Filtering (spend > 1500):
  {'county': 'NY', 'spend': 2000, 'year': 2023}
  {'county': 'NY', 'spend': 2500, 'year': 2024}

2. Projection (county, spend):
  {'county': 'LA', 'spend': 1000}
  {'county': 'NY', 'spend': 2000}
  {'county': 'LA', 'spend': 1500}
  {'county': 'NY', 'spend': 2500}

3. Group by county:
  LA: 2 documents
  NY: 2 documents

4. Aggregation (sum of spend by county):
  {'county': 'LA', 'sum(spend)': 2500}
  {'county': 'NY', 'sum(spend)': 4500}

5. Join:
  {'county': 'LA', 'population': 10000000, 'unemployment': 5.2}
  {'county': 'NY', 'population': 8000000, 'unemployment': 4.8}


## Part 4: JSON File Loading

Function to load JSON files (arrays of objects) into collections


In [4]:
# Function to load JSON file into a collection
def load_json_file(filename):
    """
    Load a JSON file (array of objects) into a Collection
    
    Args:
        filename: Path to JSON file
    
    Returns:
        Collection object
    """
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        
        # Parse JSON
        data = json_load(content)
        
        # Create collection (handle both Windows and Unix paths)
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        collection = Collection(collection_name)
        
        # Handle both array of objects and single object
        if isinstance(data, list):
            for doc in data:
                if isinstance(doc, dict):
                    collection.insert(doc)
        elif isinstance(data, dict):
            collection.insert(data)
        else:
            raise ValueError("JSON file must contain an object or array of objects")
        
        return collection
    except FileNotFoundError:
        print(f"File {filename} not found. Creating empty collection.")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)

# Function to parse CSV file into a collection
def parse_csv_line(line):
    """Parse a single CSV line, handling quoted fields"""
    fields = []
    current_field = ""
    in_quotes = False
    i = 0
    
    while i < len(line):
        char = line[i]
        
        if char == '"':
            if in_quotes and i + 1 < len(line) and line[i + 1] == '"':
                # Escaped quote
                current_field += '"'
                i += 2
            else:
                # Toggle quote state
                in_quotes = not in_quotes
                i += 1
        elif char == ',' and not in_quotes:
            # End of field
            fields.append(current_field)
            current_field = ""
            i += 1
        else:
            current_field += char
            i += 1
    
    # Add last field
    fields.append(current_field)
    return fields

def load_csv_file(filename):
    """
    Load a CSV file into a Collection
    
    Args:
        filename: Path to CSV file
    
    Returns:
        Collection object
    """
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        if len(lines) == 0:
            raise ValueError("CSV file is empty")
        
        # Parse header
        header = parse_csv_line(lines[0].strip())
        
        # Create collection
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        collection = Collection(collection_name)
        
        # Parse data rows
        for line in lines[1:]:
            line = line.strip()
            if not line:  # Skip empty lines
                continue
            
            fields = parse_csv_line(line)
            if len(fields) != len(header):
                # Skip malformed rows
                continue
            
            # Create document
            doc = {}
            for i, field in enumerate(fields):
                # Try to convert to number if possible
                field = field.strip()
                if field == '' or field == 'N/A' or field == '-9999' or field == '-8888':
                    doc[header[i]] = None
                else:
                    try:
                        # Try integer first
                        if '.' in field:
                            doc[header[i]] = float(field)
                        else:
                            doc[header[i]] = int(field)
                    except ValueError:
                        # Keep as string
                        doc[header[i]] = field
            
            collection.insert(doc)
        
        return collection
    except FileNotFoundError:
        print(f"File {filename} not found. Creating empty collection.")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)

# Function to load Excel file into a collection
def load_excel_file(filename, sheet_name=None, header_row=None, max_rows=None):
    """
    Load an Excel file into a Collection using openpyxl
    
    Args:
        filename: Path to Excel file
        sheet_name: Name of sheet to load (None for first sheet)
        header_row: Row number containing headers (0-indexed). If None, auto-detects the first non-empty row.
        max_rows: Maximum number of rows to read (including header). None for no limit. Set to 1000 for testing.
    
    Returns:
        Collection object
    """
    try:
        # Try to import openpyxl
        try:
            from openpyxl import load_workbook
        except ImportError:
            print("Warning: openpyxl not available. Install with: pip install openpyxl")
            print(f"Creating empty collection for {filename}")
            collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
            return Collection(collection_name)
        
        # Load workbook
        wb = load_workbook(filename, data_only=True)
        
        # Get sheet
        if sheet_name:
            ws = wb[sheet_name]
        else:
            ws = wb.active
        
        # Determine header row (auto-detect if not provided)
        headers = None
        current_row_index = -1
        
        def row_has_data(row):
            return any(cell is not None and str(cell).strip() != "" for cell in row)
        
        def looks_like_header_row(row):
            """Check if a row looks like a header row (has multiple short column names, not a title)"""
            non_empty_cells = [str(cell).strip() for cell in row if cell is not None and str(cell).strip() != ""]
            if len(non_empty_cells) < 2:
                return False
            
            # Check if it's a title row (one very long string or multiple cells forming a title)
            # Title rows often have text like "Table R-1. All consumer units: Annual detailed..."
            combined_text = " ".join(non_empty_cells).lower()
            if "table" in combined_text and ("annual" in combined_text or "expenditure" in combined_text or "survey" in combined_text):
                # This looks like a title/description row, not a header
                return False
            
            # Header rows typically have multiple short column names
            # Title rows often have one very long string
            avg_length = sum(len(cell) for cell in non_empty_cells) / len(non_empty_cells)
            # If average length is very long (>40 chars), it's probably a title row
            if avg_length > 40:
                return False
            
            # If we have multiple distinct short values, it's likely a header
            # Also check that we don't have too many very long cells (title-like)
            long_cells = sum(1 for cell in non_empty_cells if len(cell) > 30)
            if long_cells > len(non_empty_cells) / 2:
                return False
            
            return len(non_empty_cells) >= 2
        
        # Create collection
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        collection = Collection(collection_name)
        
        # Parse rows
        for row in ws.iter_rows(values_only=True):
            current_row_index += 1
            # Apply row limit if specified (None means no limit)
            if max_rows is not None and current_row_index >= max_rows:
                break
            # Skip rows before the specified header_row
            if header_row is not None and current_row_index < header_row:
                continue
            
            if headers is None:
                if header_row is None:
                    # Auto-detect: skip title rows, find actual header row
                    if not row_has_data(row):
                        continue  # skip empty rows
                    if not looks_like_header_row(row):
                        continue  # skip title rows
                headers = [str(cell).strip() if cell is not None else "" for cell in row]
                continue  # move to next row for data
            
            # Skip empty rows
            if all(cell is None or (isinstance(cell, str) and cell.strip() == '') for cell in row):
                continue
            
            # Create document
            doc = {}
            for i, cell_value in enumerate(row):
                if i >= len(headers):
                    continue
                field_name = headers[i].strip()
                if not field_name:
                    continue
                
                # Convert cell value
                if cell_value is None or cell_value == '':
                    doc[field_name] = None
                elif isinstance(cell_value, (int, float)):
                    doc[field_name] = cell_value
                else:
                    # Try to convert string to number
                    cell_str = str(cell_value).strip()
                    if cell_str == '' or cell_str == 'N/A' or cell_str == '-9999' or cell_str == '-8888':
                        doc[field_name] = None
                    else:
                        try:
                            if '.' in cell_str:
                                doc[field_name] = float(cell_str)
                            else:
                                doc[field_name] = int(cell_str)
                        except ValueError:
                            doc[field_name] = cell_str
            
            if doc:  # Only insert non-empty documents
                collection.insert(doc)
        
        return collection
    except FileNotFoundError:
        print(f"File {filename} not found. Creating empty collection.")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        collection_name = filename.replace('\\', '/').split('/')[-1].split('.')[0]
        return Collection(collection_name)

# Test file loading
print("Testing File Loading:")
print("=" * 50)


Testing File Loading:


## Part 5: Sample Data Generation

Creating sample datasets for Green Purchasing Behavior analysis


In [5]:
# Note: Food spending data comes from consumer expenditure surveys
# Employment data comes from employment Excel files
# No proxies or Food Environment Atlas data used

# Transform Excel data files for Green Purchasing Behavior analysis

# Helper utilities for Excel -> Collection transformations
def get_field_value(doc, keywords):
    """Get field value by matching keywords in column names (handles newlines in column names)"""
    for key, value in doc.items():
        if key is None:
            continue
        # Normalize key: remove newlines, convert to lowercase
        key_normalized = str(key).replace('\n', ' ').replace('\r', ' ').strip().lower()
        for keyword in keywords:
            if keyword in key_normalized:
                if isinstance(value, str):
                    return value.strip()
                return value
    return None

def to_float(value):
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    value_str = str(value).strip()
    if value_str == "":
        return None
    value_str = value_str.replace(',', '')
    if value_str.endswith('%'):
        value_str = value_str[:-1]
    try:
        return float(value_str)
    except ValueError:
        return None

def to_int(value):
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return int(value)
    value_str = str(value).strip()
    if value_str == "":
        return None
    value_str = value_str.replace(',', '')
    match = re.search(r'-?\d+', value_str)
    if match:
        try:
            return int(match.group(0))
        except ValueError:
            return None
    return None

def infer_year(value, default_year):
    if value is None:
        return default_year
    if isinstance(value, (int, float)):
        year = int(value)
        if 1900 <= year <= 2100:
            return year
    value_str = str(value)
    match = re.search(r'(19|20)\d{2}', value_str)
    if match:
        return int(match.group(0))
    return default_year

def transform_consumer_data(consumer_collection):
    """
    Transform consumer spending data from Excel into food spending format.
    Consumer Expenditure Surveys are typically national-level, so we use "national" as default county.
    """
    food_spending = Collection("food_spending")
    county_keywords = ['county', 'area', 'region', 'location', 'metro', 'city', 'borough', 'urban', 'geography', 'geographic']
    category_keywords = ['item', 'category', 'product', 'series', 'description', 'class', 'item_name', 'item_name_1', 'item_name_2', 'expenditure']
    spend_keywords = ['expenditure', 'spend', 'value', 'amount', 'dollar', 'sales', 'cost', 'price', 'avg', 'average', 'mean', 'cu', 'consumer', 'annual', 'total', 'units', 'all']
    year_keywords = ['year', 'date', 'period', 'month', 'time', 'survey']
    
    processed = 0
    skipped_no_spend = 0
    skipped_not_food = 0
    
    for doc in consumer_collection:
        # Try to find county/geographic identifier, default to "national" for consumer expenditure surveys
        county = get_field_value(doc, county_keywords)
        if not county:
            county = "national"  # Consumer expenditure surveys are typically national-level
        
        # Find category/item name - look for "Item" field first
        category = None
        if 'Item' in doc:
            category = str(doc['Item']).strip()
        else:
            category = get_field_value(doc, category_keywords)
        
        if not category:
            # Try to use any field that looks like a description
            for key, value in doc.items():
                if key and 'item' in str(key).lower() and value and isinstance(value, str) and len(value) > 3:
                    category = str(value).strip()
                    break
        
        if not category:
            skipped_not_food += 1
            continue
        
        # Find spending/expenditure value - check all numeric columns
        spend = None
        
        # First try keyword matching
        spend = to_float(get_field_value(doc, spend_keywords))
        
        # If not found, look for numeric columns (excluding Item)
        # Handle "All\nconsumer\nunits" column which contains spending values
        if spend is None:
            for key, value in doc.items():
                if key and 'item' not in str(key).lower() and value is not None:
                    # Try to convert to float
                    num_value = to_float(value)
                    if num_value is not None and num_value > 0:
                        spend = num_value
                        break
        
        if spend is None or spend <= 0:
            skipped_no_spend += 1
            continue
        
        year = infer_year(get_field_value(doc, year_keywords), 2023)
        
        # Only include food-related categories
        category_lower = str(category).strip().lower()
        food_keywords = ['food', 'grocery', 'restaurant', 'dining', 'meal', 'beverage', 'drink', 'organic', 'fresh', 'vegetable', 'fruit', 'meat', 'dairy', 'bakery', 'cereal', 'cereals', 'bakery products', 'beef', 'pork', 'poultry', 'seafood', 'fish', 'milk', 'cheese', 'eggs', 'bread', 'rice', 'pasta', 'snacks', 'sugar', 'coffee', 'tea', 'juice', 'soda', 'alcoholic', 'expenditure', 'spending', 'expenses', 'cost', 'price']
        
        # Check if category contains food-related keywords
        is_food_related = any(keyword in category_lower for keyword in food_keywords)
        
        # Also check for non-food items to exclude (more specific exclusions)
        non_food_keywords = ['consumer units', 'number of consumer', 'consumer unit characteristics', 'income before taxes', 'age', 'education', 'race', 'housing', 'transportation', 'health', 'insurance', 'pension', 'social security', 'tax', 'mortgage', 'rent', 'utilities', 'telephone', 'water', 'sewer', 'trash', 'fuel', 'furniture', 'appliances', 'clothing', 'shoes', 'jewelry', 'personal care', 'reading', 'miscellaneous', 'cash contributions', 'gifts', 'entertainment', 'recreation', 'sports', 'pets', 'toys', 'games', 'hobbies', 'characteristics']
        
        is_non_food = any(keyword in category_lower for keyword in non_food_keywords)
        
        # Be more lenient - if it's not clearly non-food and has a spend value, include it
        if is_non_food:
            skipped_not_food += 1
            continue
        
        food_spending.insert({
            "county": str(county),
            "category": category_lower.replace(' ', '_').replace('/', '_').replace('(', '').replace(')', '').replace(',', ''),
            "spend": spend,
            "year": year
        })
        processed += 1
    
    return food_spending

def transform_employment_excel_data(employment_collection):
    """
    Transform employment data from Excel into jobs/employment format
    """
    jobs = Collection("jobs")
    county_keywords = ['county', 'area', 'region', 'location', 'metro', 'city', 'borough']
    income_keywords = ['median_income', 'income', 'wage', 'salary', 'earnings', 'pay']
    occupation_keywords = ['occupation', 'job', 'title', 'category', 'sector', 'industry', 'class']
    year_keywords = ['year', 'date', 'period', 'month']
    
    for doc in employment_collection:
        county = get_field_value(doc, county_keywords)
        if not county:
            continue
        median_income = to_int(get_field_value(doc, income_keywords))
        occupation = get_field_value(doc, occupation_keywords) or "general"
        year = infer_year(get_field_value(doc, year_keywords), 2024)
        
        jobs.insert({
            "county": str(county),
            "occupation": str(occupation).strip().lower().replace(' ', '_'),
            "median_income": median_income,
            "year": year
        })
    
    return jobs

def transform_unemployment_excel_data(unemployment_collection):
    """
    Transform unemployment data from Excel into unemployment format.
    Handles data where unemployment rates are in year columns (2020, 2021, etc.)
    """
    unemployment = Collection("unemployment")
    county_keywords = ['county', 'area', 'region', 'location', 'metro', 'city', 'borough', 'msa', 'metropolitan', 'geography', 'geographic', 'title']
    
    processed = 0
    skipped_no_county = 0
    
    for doc in unemployment_collection:
        # Find county/area - look for "Metropolitan area title" or similar
        county = get_field_value(doc, county_keywords)
        
        # If not found by keywords, look for any text field that might be a location
        if not county:
            for key, value in doc.items():
                if key and value:
                    key_normalized = str(key).replace('\n', ' ').lower()
                    # Skip numeric and date fields, but include "title" fields
                    if any(kw in key_normalized for kw in ['fips', 'code', 'rate', 'percent', 'pct', 'unemployment', 'labor']) and 'title' not in key_normalized:
                        continue
                    if isinstance(value, str) and len(value) > 2 and len(value) < 100:
                        # Check if it's not a number
                        try:
                            float(str(value))
                        except ValueError:
                            county = str(value).strip()
                            break
        
        if not county:
            skipped_no_county += 1
            continue
        
        # Extract unemployment rates from year columns (2020, 2021, 2022, 2023, 2024)
        # Create one record per year
        for key, value in doc.items():
            if key is None:
                continue
            
            # Check if key is a year (2020-2024)
            key_str = str(key).strip()
            if key_str.isdigit():
                year = int(key_str)
                if 2020 <= year <= 2024:
                    rate = to_float(value)
                    if rate is not None and 0 <= rate <= 100:  # Reasonable rate range
                        unemployment.insert({
                            "county": str(county),
                            "rate": rate,
                            "year": year
                        })
                        processed += 1
    
    return unemployment

print("Excel data transformation functions created")


Excel data transformation functions created


## Part 6: Green Purchasing Behavior Application

Application that uses all implemented functions to analyze sustainable food purchasing behavior


In [6]:
# Green Purchasing Behavior Analysis Application

print("=" * 70)
print("GREEN PURCHASING BEHAVIOR ANALYSIS APPLICATION")
print("=" * 70)

# Load data from Excel files
print("\n1. Loading Data from Excel Files:")
print("-" * 70)

# Load consumer spending data
print("Loading consumer spending data...")
consumer_data_raw = load_excel_file('consumer-data/cu-all-detail-2023.xlsx')#, max_rows=200)
print(f"Loaded {len(consumer_data_raw)} records from consumer data")

# Load employment data
print("Loading employment data...")
employment_data_raw = load_excel_file('employement_data/all_data_M_2024.xlsx')#, max_rows=1000)
print(f"Loaded {len(employment_data_raw)} records from employment data")

# Load unemployment data
print("Loading unemployment data...")
unemployment_data_raw = load_excel_file('unemployment-rate-data/metro-annual-unemployment-rates.xlsx')#, max_rows=1000)
print(f"Loaded {len(unemployment_data_raw)} records from unemployment data")



# Transform Excel data for analysis
print("\n2. Transforming Data for Green Purchasing Analysis:")
print("-" * 70)

# Transform consumer data to food spending format
food_spending = transform_consumer_data(consumer_data_raw)
print(f"Transformed to {len(food_spending)} food spending records from consumer data")

# Transform employment data
jobs = transform_employment_excel_data(employment_data_raw)
print(f"Transformed to {len(jobs)} job/employment records from employment data")

# Transform unemployment data
unemployment = transform_unemployment_excel_data(unemployment_data_raw)
print(f"Transformed to {len(unemployment)} unemployment records from unemployment data")



# Application Question 1: Who is buying sustainable food? (Demographics, Geography)
print("\n3. Question 1: Who is buying sustainable food? (Geography Analysis)")
print("-" * 70)

# Filter: High spending counties (spend > 1000)
print("\n2a. Filtering: Counties with spending > $1000")
high_spending = filter_collection(food_spending, lambda doc: doc.get("spend", 0) > 1000)
print(f"Found {len(high_spending)} records with spending > $1000")

# Projection: County and spend
print("\n2b. Projection: County and spending amounts")
county_spending = project_collection(high_spending, ["county", "spend"])
for doc in county_spending:
    print(f"  {doc['county']}: ${doc['spend']:.2f}")

# Group by: County
print("\n2c. Group By: Spending by county")
county_groups = group_by(food_spending, "county")
for county, docs in county_groups.items():
    total = sum(doc.get("spend", 0) for doc in docs)
    print(f"  {county}: ${total:.2f} total spending ({len(docs)} records)")

# Aggregation: Total spending by county
print("\n2d. Aggregation: Total spending by county")
total_by_county = aggregate(food_spending, "county", "spend", "sum")
for result in total_by_county:
    print(f"  {result['county']}: ${result['sum(spend)']:.2f}")

# Application Question 2: How income influences sustainable purchasing
print("\n4. Question 2: How income influences sustainable purchasing behavior")
print("-" * 70)

# Note: Consumer expenditure data is national-level ("national" county), while employment data is county-level
# Join would return 0 records due to county name mismatch - this is expected and shows data granularity differences
print("\n4a. Attempted Join: Food spending with jobs data (on county)")
print("Note: Consumer expenditure data uses 'national' as county (aggregated national data)")
print("      Employment data uses actual county/metro area names")
spending_jobs = join_collections(food_spending, jobs, "county", "county")
print(f"Joined collection has {len(spending_jobs)} records")
print("Explanation: No records because 'national' ≠ actual county names.")
print("This is GOOD - it shows the data sources have different granularities:")
print("  - Consumer data: National averages (what people spend overall)")
print("  - Employment data: County/metro level (where jobs are located)")
print("  - This separation allows us to analyze each dataset independently")

# Analyze employment data separately to understand income distribution
print("\n4b. Analysis: Income distribution in employment data (separate analysis)")
def categorize_income(doc):
    income = doc.get("median_income", 0) or 0
    if income >= 75000:
        return "very_high"
    elif income >= 50000:
        return "high"
    elif income >= 35000:
        return "medium"
    else:
        return "low"

jobs_with_category = Collection("jobs_categorized")
for doc in jobs:
    new_doc = doc.copy()
    new_doc["income_category"] = categorize_income(doc)
    jobs_with_category.insert(new_doc)

income_distribution = aggregate(jobs_with_category, "income_category", "median_income", "count")
print("Employment records by income category:")
for result in income_distribution:
    print(f"  {result['income_category']}: {result['count(median_income)']} counties/metro areas")

# Analyze food spending separately
print("\n4c. Analysis: National food spending patterns (separate analysis)")
avg_food_spending = aggregate(food_spending, "category", "spend", "avg")
print("Top 10 food categories by average spending:")
sorted_food = sorted(avg_food_spending, key=lambda x: x.get('avg(spend)', 0), reverse=True)
for result in sorted_food[:10]:
    print(f"  {result['category']}: ${result['avg(spend)']:.2f}")



# Application Question 3: Economic shocks and spending habits
print("\n5. Question 3: Do economic shocks (unemployment) change spending habits?")
print("-" * 70)

# Analyze unemployment trends over time
print("\n5a. Analysis: Unemployment trends by year (separate analysis)")
year_unemployment = aggregate(unemployment, "year", "rate", "avg")
print("Average unemployment rate by year:")
for result in sorted(year_unemployment, key=lambda x: x.get('year', 0)):
    print(f"  {result['year']}: {result['avg(rate)']:.2f}%")

# Analyze unemployment by category
print("\n5b. Analysis: Unemployment rate distribution")
def categorize_unemployment(doc):
    rate = doc.get("rate", 0)
    if rate >= 8.0:
        return "high"
    elif rate >= 5.0:
        return "medium"
    else:
        return "low"

unemployment_categorized = Collection("unemployment_categorized")
for doc in unemployment:
    new_doc = doc.copy()
    new_doc["unemployment_category"] = categorize_unemployment(doc)
    unemployment_categorized.insert(new_doc)

unemployment_dist = aggregate(unemployment_categorized, "unemployment_category", "rate", "count")
print("Metro areas by unemployment level:")
for result in unemployment_dist:
    print(f"  {result['unemployment_category']}: {result['count(rate)']} metro areas")

# Analyze food spending by year
print("\n5c. Analysis: Food spending trends by year (separate analysis)")
year_food_spending = aggregate(food_spending, "year", "spend", "avg")
print("Average food spending by year:")
for result in sorted(year_food_spending, key=lambda x: x.get('year', 0)):
    print(f"  {result['year']}: ${result['avg(spend)']:.2f}")

print("\n5d. Insight: Compare unemployment trends (5a) with food spending trends (5c)")
print("  - Rising unemployment may correlate with changes in food spending patterns")
print("  - National food spending data shows overall consumer behavior")
print("  - Metro-level unemployment shows regional economic conditions")

# Application Question 4: Employment and green purchasing relationship
print("\n6. Question 4: How does employment status affect green purchasing behavior?")
print("-" * 70)

# Analyze employment data separately
print("\n6a. Analysis: Employment distribution by occupation type")
occupation_dist = aggregate(jobs, "occupation", "median_income", "count")
print("Counties/metro areas by occupation type:")
for result in sorted(occupation_dist, key=lambda x: x.get('count(median_income)', 0), reverse=True):
    print(f"  {result['occupation']}: {result['count(median_income)']} areas")

# Analyze income levels in employment data
print("\n6b. Analysis: Income distribution in employment data")
income_ranges = aggregate(jobs, "median_income", "occupation", "count")
print("Top 10 income levels by number of areas:")
sorted_income = sorted(income_ranges, key=lambda x: x.get('count(occupation)', 0), reverse=True)
for result in sorted_income[:10]:
    if result.get('median_income'):
        print(f"  ${result['median_income']:,}: {result['count(occupation)']} areas")

# Analyze food spending categories
print("\n6c. Analysis: Food spending categories (national level)")
food_categories = aggregate(food_spending, "category", "spend", "sum")
# Filter out non-food categories (statistical measures)
food_keywords_filter = ['mean', 'se', 'rse', 'people_', 'children_', 'adults_', 'earners_', 'vehicles_', 'men_', 'women_', 'homeowner_', 'black_', 'white_', 'asian_', 'hispanic_', 'elementary_', 'high_school_', 'college_', 'at_least_one_', 'percent_reporting']
actual_food_categories = []
for result in food_categories:
    category = result.get('category', '')
    is_food = not any(keyword in category.lower() for keyword in food_keywords_filter)
    if is_food and result.get('sum(spend)', 0) > 0:
        actual_food_categories.append(result)

print("Top 15 actual food spending categories:")
sorted_food = sorted(actual_food_categories, key=lambda x: x.get('sum(spend)', 0), reverse=True)
for result in sorted_food[:15]:
    print(f"  {result['category']}: ${result['sum(spend)']:.2f}")

# Application Question 5: Category analysis
print("\n7. Question 5: Spending patterns by food category")
print("-" * 70)

# Filter out non-food categories (statistical measures, demographics)
food_keywords_filter = ['mean', 'se', 'rse', 'people_', 'children_', 'adults_', 'earners_', 'vehicles_', 'men_', 'women_', 'homeowner_', 'black_', 'white_', 'asian_', 'hispanic_', 'elementary_', 'high_school_', 'college_', 'at_least_one_', 'percent_reporting']

# Filter food spending to actual food categories
actual_food_spending = Collection("actual_food_spending")
for doc in food_spending:
    category = doc.get("category", "")
    is_food = not any(keyword in category.lower() for keyword in food_keywords_filter)
    if is_food:
        actual_food_spending.insert(doc)

print(f"Filtered to {len(actual_food_spending)} actual food spending records (removed {len(food_spending) - len(actual_food_spending)} statistical/demographic records)")

# Aggregation: Total spending by food category
print("\n7a. Aggregation: Total spending by food category")
total_by_category = aggregate(actual_food_spending, "category", "spend", "sum")
sorted_total = sorted(total_by_category, key=lambda x: x.get('sum(spend)', 0), reverse=True)
print("Top 20 food categories by total spending:")
for result in sorted_total[:20]:
    print(f"  {result['category']}: ${result['sum(spend)']:.2f}")

# Aggregation: Average spending by food category
print("\n7b. Aggregation: Average spending per record by food category")
avg_by_category = aggregate(actual_food_spending, "category", "spend", "avg")
sorted_avg = sorted(avg_by_category, key=lambda x: x.get('avg(spend)', 0), reverse=True)
print("Top 20 food categories by average spending:")
for result in sorted_avg[:20]:
    print(f"  {result['category']}: ${result['avg(spend)']:.2f}")

# Complex query: Category spending analysis
print("\n7c. Complex Query: Food category distribution")
category_counts = aggregate(actual_food_spending, "category", "spend", "count")
print(f"Total unique food categories: {len(category_counts)}")
print("Categories with highest record counts:")
sorted_counts = sorted(category_counts, key=lambda x: x.get('count(spend)', 0), reverse=True)
for result in sorted_counts[:10]:
    print(f"  {result['category']}: {result['count(spend)']} records")

print("\n" + "=" * 70)
print("APPLICATION ANALYSIS COMPLETE")
print("=" * 70)


GREEN PURCHASING BEHAVIOR ANALYSIS APPLICATION

1. Loading Data from Excel Files:
----------------------------------------------------------------------
Loading consumer spending data...
Loaded 4459 records from consumer data
Loading employment data...
Loaded 414437 records from employment data
Loading unemployment data...
Loaded 391 records from unemployment data

2. Transforming Data for Green Purchasing Analysis:
----------------------------------------------------------------------
Transformed to 2543 food spending records from consumer data
Transformed to 414437 job/employment records from employment data
Transformed to 1935 unemployment records from unemployment data

3. Question 1: Who is buying sustainable food? (Geography Analysis)
----------------------------------------------------------------------

2a. Filtering: Counties with spending > $1000
Found 116 records with spending > $1000

2b. Projection: County and spending amounts
  national: $101805.00
  national: $1883.42
  

## Summary

This project implements:

1. **Extended JSON Parser**: Handles objects, arrays, nested structures, booleans, and null values
2. **CSV Parser**: Custom CSV parser that handles quoted fields and converts data types appropriately
3. **Excel Parser**: Custom Excel file parser using openpyxl to load .xlsx files into collections
4. **Collection Class**: Stores and manages JSON documents (similar to MongoDB collections)
5. **Core Operations**:
   - **Filtering**: Select documents based on conditions
   - **Projection**: Select specific fields from documents
   - **Group By**: Group documents by a key
   - **Aggregation**: Compute aggregates (sum, avg, max, min, count) on grouped data
   - **Join**: Join two collections on specified keys

6. **Data Transformation**: Functions to transform real-world data into format suitable for green purchasing analysis:
   - **Consumer Data**: Transforms consumer spending Excel data into food spending format
   - **Employment Data**: Transforms employment Excel data into jobs/employment format
   - **Unemployment Data**: Transforms unemployment Excel data into unemployment rate format
   - Maps to green purchasing behavior indicators

7. **Application**: Green Purchasing Behavior analysis using real data from Excel files:
   - **Consumer spending data** from Excel files (cu-all-detail-2023.xlsx)
   - **Employment data** from Excel files (all_data_M_2024.xlsx)
   - **Unemployment data** from Excel files (metro-annual-unemployment-rates.xlsx)
   - Analyzes who buys sustainable food (geography)
   - Examines income influence on purchasing behavior
   - **Analyzes employment status impact** on green purchasing (using actual employment data, no proxies)
   - Studies economic shocks (unemployment/poverty) impact on spending
   - Analyzes spending patterns by food category

All operations are implemented from scratch without using pandas, json, or csv libraries. The project uses real data from Excel files only - no proxies or synthetic data.
